In [1]:
import xml.etree.ElementTree as ET
import os
import dill
import pandas as pd
from datetime import datetime
import operator
import re

In [2]:
os.chdir('/home/vagrant/projects/stackoverflow-post')
posts = dill.load(open('data/posts.pkd', 'r'))

In [7]:
# xml = ET.iterparse('Posts.xml', events = ('start', 'end'))
# # PostTypeId, CreationDate, 
# posts = []
# for event, elem in iter(xml):
#     if len(posts) < 500000:  
#         tag = elem.tag
#         value = elem.attrib
#         if event == 'start':
#             if tag=='row':  
#                 d = dict()
#                 if 'OwnerUserId' in value:
#                     d['OwnerUserId'] = value['OwnerUserId']
#                 elif 'OwnerDisplayName' in value:
#                     d['OwnerUserId'] = value['OwnerDisplayName']
#                 else:
#                     d['OwnerUserId'] = None
#                 if value['PostTypeId']=='1':
#                     d['Tags'] = value['Tags']
#                     d['ParentId'] = None
#                     d['Id'] = value['Id']
#                     d['CreationDate'] = datetime.strptime(value['CreationDate'], '%Y-%m-%dT%H:%M:%S.%f')
#                     d['Body'] = value['Body']
#                     d['Title'] = value['Title']
#                     d['Score'] = value['Score']
#                     d['PostTypeId'] = value['PostTypeId']
#                 elif value['PostTypeId']=='2':
# #                     d['Tags'] = value['Tags']
#                     d['ParentId'] = value['ParentId']
#                     d['Id'] = value['Id']
#                     d['CreationDate'] = datetime.strptime(value['CreationDate'], '%Y-%m-%dT%H:%M:%S.%f')
#                     d['Body'] = value['Body']
#                     d['Title'] = None
#                     d['Score'] = value['Score']
#                     d['PostTypeId'] = value['PostTypeId']
#                 posts.append(d)
#         elem.clear()
#     else:
#         break

# dill.dump(posts, open('data/posts.pkd', 'w'))
# root = doc.getroot()

In [8]:
names = list(posts[0].keys())
df = pd.DataFrame(posts, columns = names)
subnetwork = df.loc[:,['OwnerUserId', 'ParentId', 'PostTypeId']]
dill.dump(subnetwork, open('data/subnetwork.pkd', 'w'))

In [9]:
byOwnerID = df.groupby('OwnerUserId')
#Questions raised by owner342
# Owner342 = byOwnerID.get_group('342')
# Owner342.loc[(Owner342.PostTypeId == '1'), 'Id']
#Answers posted by Owner342
# Owner342.loc[(Owner342.PostTypeId == '2'), 'Id'].head()
byQuestionID = df.groupby('ParentId')

In [11]:
#for each user (key), group together questions asked, and answers received for that question
Users = dict()
for key, value in byOwnerID.groups.items():
    QA = []
    Owner = byOwnerID.get_group(key)
    QuestionIDs = Owner.loc[(Owner.PostTypeId == '1'), 'Id']
    for QuestionID in QuestionIDs:
        qa = dict()
        if QuestionID in byQuestionID.groups:
            q = Owner.loc[(Owner.Id == QuestionID),]
            a = byQuestionID.get_group(QuestionID).sort_values('CreationDate')
            qa['title'] = q['Title'].tolist()
            qa['body'] = q['Body'].tolist()
            qa['date'] = q['CreationDate'].tolist()
            qa['tag'] = q['Tags'].tolist()
            qa['timeline'] = a['CreationDate'].tolist()
            qa['score'] = a['Score'].tolist()
            QA.append(qa) 
    Users[key] = QA

In [17]:
# Collect characteristics for each question:
# Question characteristics: tag, body text, body length, title, date of post, score of post
# Answer characteristics: first answer score, time till first answer, 
# time till first answer scoring > 5 (10,20,50)
QChar = []
for key, value in Users.items():
    if value:
        for i in value:
            d = dict()
            d['q_owner'] = key
            d['q_tag'] = i['tag'][0]
            d['q_body'] = i['body'][0]
            d['q_body_len'] = len(list(i['body'][0]))
            d['q_title'] = i['title'][0]
            d['q_date'] = i['date'][0]
            d['q_hour'] = datetime.strftime(i['date'][0], '%H')
            d['q_day'] = datetime.strftime(i['date'][0], '%A')
            d['q_score'] = int(i['score'][0])
            
            if len(i['score'])>1:
                d['fa_score'] = int(i['score'][1])
                d['fa_time'] = i['timeline'][1]-i['timeline'][0]
                d['fa_time'] = d['fa_time'].total_seconds()
            
            d['s5_time'] = None
            d['s10_time'] = None
            d['s20_time'] = None
            d['s50_time'] = None
            find5 = False
            find10 = False
            find20 = False
            find50 = False
            j=1
            while j<len(i['score']):
                if (int(i['score'][j]) >=5) and find5==False:
                    d['s5_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s5_time'] = d['s5_time'].total_seconds()
                    find5 = True
                elif (int(i['score'][j]) >=10) and find10==False:
                    d['s10_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s10_time'] = d['s10_time'].total_seconds()
                    find10 = True
                elif (int(i['score'][j]) >=20) and find20==False:
                    d['s20_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s20_time'] = d['s20_time'].total_seconds()
                    find20 = True
                elif (int(i['score'][j]) >=50) and find50==False:
                    d['s50_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s50_time'] = d['s50_time'].total_seconds()
                    find50 = True
                else:
                    j+=1
            QChar.append(d)

In [18]:
# import nltk
# from bs4 import BeautifulSoup as bs
df = pd.DataFrame(QChar)
df.q_date = pd.to_datetime(df.q_date)

In [19]:
df #18670 posts

,fa_score,fa_time,q_body,q_body_len,q_date,q_day,q_hour,q_owner,q_score,q_tag,q_title,s10_time,s20_time,s50_time,s5_time
0,4.0,434.554,<p>I am thinking along the lines of replicatin...,367,2008-09-19 05:09:25.977,Friday,05,essexboyracer,0,<php>,Convert/extract phpinfo() into php.ini,NaN,NaN,NaN,NaN
1,2.0,11504.657,<p>Has anyone any reources for learning how to...,402,2008-09-24 06:42:46.107,Wednesday,06,essexboyracer,2,<php><mysql><svg><vector-graphics><php-gtk>,Interactive SVG - Learning Resources?,NaN,NaN,NaN,NaN
2,1.0,1630433.760,<p>I'm trying to decode a WBXML encoded SyncML...,632,2008-10-22 15:25:00.777,Wednesday,15,11542,1,<python><s60><syncml><wbxml>,Decoding a WBXML SyncML message from an S60 de...,NaN,NaN,NaN,NaN
3,110.0,5496.290,<p>I want to get the headers only from a curl ...,401,2008-11-13 14:00:43.840,Thursday,14,11542,30,<curl>,Curl post data and headers only,5496.290,5496.290,5496.290,5496.290
4,NaN,NaN,<p>I'm just starting to try out phpunit on som...,651,2008-11-14 10:37:13.877,Friday,10,11542,7,<php><unit-testing><phpunit>,PHPUnit filenaming conventions,NaN,NaN,NaN,NaN
5,6.0,54503.460,<p>How can UTF-8 strings (i.e. 8-bit string) b...,975,2008-09-22 23:55:02.883,Monday,23,11543,0,<php><html><xml><unicode><utf-8>,Convert a UTF-8 string to/from 7-bit XML in PHP,NaN,NaN,NaN,54503.460
6,20.0,455.910,<p>JavaScript allows functions to be treated a...,749,2008-09-23 22:30:54.573,Tuesday,22,11543,0,<javascript><function><properties>,"How to convert an ""object"" into a function in ...",455.910,455.910,NaN,455.910
7,21.0,343.496,<p>What's the fastest way to count the number ...,381,2008-09-24 08:56:21.263,Wednesday,08,11543,1,<javascript><performance><properties><count><key>,How to efficiently count the number of keys/pr...,343.496,343.496,NaN,343.496
8,1.0,1250487.416,"<p>In <code>Eclipse PDT</code>, <code>Ctrl-Shi...",984,2008-09-25 10:27:52.750,Thursday,10,11543,2,<php><eclipse><comments><eclipse-pdt><reformat>,How to reformat multi-line comments in Eclipse...,NaN,NaN,NaN,NaN
9,36.0,172.693,<p>How do you do case conversion in XSL?</p>\n...,184,2009-02-25 14:43:02.327,Wednesday,14,11543,0,<string><xpath><xslt><case-conversion>,How can I convert a string to upper- or lower-...,172.693,172.693,298.773,172.693


In [20]:
df.describe()

,fa_score,fa_time,q_body_len,q_score,s10_time,s20_time,s50_time,s5_time
count,86040.000000,8.604000e+04,108831.000000,108831.000000,2.359400e+04,1.337000e+04,6.038000e+03,3.968700e+04
mean,10.239017,2.054140e+05,826.968548,12.202130,2.933400e+05,3.374605e+05,3.804551e+05,2.534745e+05
std,87.144389,1.112100e+06,897.134121,104.834126,1.493583e+06,1.621331e+06,1.748377e+06,1.334165e+06
min,-54.000000,0.000000e+00,18.000000,-32.000000,3.000000e-02,3.000000e-02,4.700000e-02,0.000000e+00
25%,1.000000,1.178423e+02,342.000000,1.000000,1.706360e+02,1.868118e+02,1.885143e+02,1.573520e+02
50%,2.000000,5.381635e+02,598.000000,2.000000,7.371380e+02,8.314265e+02,8.532100e+02,6.807700e+02
75%,5.000000,5.137934e+03,1018.000000,6.000000,5.925162e+03,6.689176e+03,7.372269e+03,5.328890e+03
max,12198.000000,1.961131e+07,34183.000000,14404.000000,1.884576e+07,1.951967e+07,1.951967e+07,1.847472e+07


In [2]:
# dill.dump(df, open('data/df.pkd', 'w'))
df = dill.load(open('data/df.pkd', 'r'))

In [4]:
import unicodedata
for i in posts:
    if type(i['Body']) == unicode:
        i['Body'] = unicodedata.normalize('NFKD', i['Body']).encode('ascii', 'ignore')
    if type(i['OwnerUserId']) == unicode:
        i['OwnerUserId'] = unicodedata.normalize('NFKD', i['OwnerUserId']).encode('ascii', 'ignore')
output = [(re.sub('\n', '', i['Body']), i['OwnerUserId'], i['CreationDate']) for i in posts]
pd.DataFrame(output[0:100], columns=['html', 'id', 'creationdate']).to_csv('data/exampleText.csv')